# How to implement RAG Pattern using Semantic Kernel and Azure Cognitive Search
**NOTE**: This notebook requires that a search index exists with semantic search and vector index enabled.

Follow the steps in the [Notebook](https://github.com/fsaleemm/cognitive-search-vector-pr/blob/main/demo-python/code/azure-search-vector-python-sample.ipynb) to create the vector index with semantic search enabled.

In [ ]:
!python -m pip install python-dotenv==1.0.0
!python -m pip install --upgrade semantic-kernel
!python -m pip install azure-search-documents==11.4.0b9

In [1]:
import os, json

from dotenv import load_dotenv
if not load_dotenv(): raise Exception(".env file not found")

### Setup Semantic Kernel

In [2]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding

kernel = Kernel()

kernel.add_text_embedding_generation_service(
    "ada",
    AzureTextEmbedding(
        os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
        os.getenv("AZURE_OPENAI_ENDPOINT"),
        os.getenv("AZURE_OPENAI_API_KEY"),
    ),
)

kernel.add_chat_service(
    "chat",
    AzureChatCompletion(
        os.getenv("AZURE_OPENAI_CHAT_MODEL"),
        os.getenv("AZURE_OPENAI_ENDPOINT"),
        os.getenv("AZURE_OPENAI_API_KEY"),
    ),
)

### Get the intent of the question being asked

In [3]:
question = "What services are best for asynchornous communications?"

In [4]:
#intent detection

from semantic_kernel import PromptTemplate, PromptTemplateConfig, SemanticFunctionConfig


prompt = """Bot: How can I help you?
User: {{$input}}

---------------------------------------------

The intent of the user in 5 words or less: """

prompt_config = PromptTemplateConfig(
    description="Gets the intent of the user.",
    type="completion",
    completion=PromptTemplateConfig.CompletionConfig(0.0, 0.0, 0.0, 0.0, 500),
    input=PromptTemplateConfig.InputConfig(
        parameters=[
            PromptTemplateConfig.InputParameter(
                name="input", description="The user's request.", default_value=""
            )
        ]
    ),
)


# Create the SemanticFunctionConfig object
prompt_template = PromptTemplate(
    template=prompt,
    template_engine=kernel.prompt_template_engine,
    prompt_config=prompt_config,
)

function_config = SemanticFunctionConfig(prompt_config, prompt_template)

get_intent = kernel.register_semantic_function(
    skill_name="OrchestratorPlugin",
    function_name="GetIntent",
    function_config=function_config,
)

result_intent = await kernel.run_async(
    get_intent,
    input_str=question,
)

print(result_intent.result)

Inquiring about asynchronous communication services


### Get relevant data from the ACS index using hybrid search

In [5]:
import os, time
import openai 

openai.api_type = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  

# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="text-embedding-ada-002")
    embeddings = response['data'][0]['embedding']
    return embeddings

In [6]:
#get relevant data for context

from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, IndexDocumentsBatch  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector 
from azure.search.documents.models import QueryType

service_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")  
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")  
key = os.getenv("AZURE_SEARCH_API_KEY")  
credential = AzureKeyCredential(key)

search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
vector = Vector(value=generate_embeddings(result_intent.result), 
                k=3, 
                fields="contentVector" # These may need to be adjusted based on your index configuration
            )

results_context = search_client.search(  
    search_text=result_intent, 
    vectors= [vector],
    select=["title", "content", "category"], # These may need to be adjusted based on your index configuration
    query_type=QueryType.SEMANTIC, 
    query_language="en-us", 
    semantic_configuration_name='my-semantic-config', # These may need to be adjusted based on your index configuration
    query_caption="extractive", 
    query_answer="extractive|count-2",
    top=3
)  

results_content = []

for r in results_context:
    print(r["content"])
    results_content.append(r["content"])

results =" ".join(results_content)

Azure Service Bus is a fully managed, enterprise-grade messaging service that enables you to build reliable and scalable applications. It provides features like message queuing, publish-subscribe, and dead-lettering. Service Bus supports various messaging patterns, including point-to-point, broadcast, and request-reply. You can use Service Bus to integrate your applications and services, decouple your system components, and handle asynchronous communication. It also integrates with other Azure services, such as Azure Functions and Azure Logic Apps.
Azure Queue Storage is a fully managed, scalable, and durable message queuing service that enables you to decouple your applications and build asynchronous solutions. It provides features like at-least-once delivery, message time-to-live, and a RESTful API. Queue Storage supports various programming languages, such as C#, Java, and Python. You can use Azure Queue Storage to build distributed applications, offload your databases, and process 

### Summarize the result

In [7]:
#Summarize

import semantic_kernel


sprompt = """
Considering these facts

Facts: {{$results}}

Question: {{$input}}

Provide a concise answer ('Answer: ') and a separate explanation ('Explanation: '), in two lines.
"""

sprompt_config = PromptTemplateConfig(
    description="Gets the intent of the user.",
    type="completion",
    completion=PromptTemplateConfig.CompletionConfig(0.5, 0.0, 0.0, 0.0, 1024),
    input=PromptTemplateConfig.InputConfig(
        parameters=[
            PromptTemplateConfig.InputParameter(
                name="input", description="The user's request.", default_value=""
            ),
            PromptTemplateConfig.InputParameter(
                name="results", description="The result from grounding data", default_value=""
            )
        ]
    ),
)


# Create the SemanticFunctionConfig object
sprompt_template = PromptTemplate(
    template=sprompt,
    template_engine=kernel.prompt_template_engine,
    prompt_config=sprompt_config,
)

sfunction_config = SemanticFunctionConfig(sprompt_config, sprompt_template)

get_summary = kernel.register_semantic_function(
    skill_name="OrchestratorPlugin",
    function_name="GetSummary",
    function_config=sfunction_config,
)

variables = semantic_kernel.ContextVariables()
variables["input"] = question
variables["results"] = results

result_summary = await kernel.run_async(
    get_summary,
    input_vars=variables
)

print(f"Original Question: {question}")
print(f"Optimized Question: {result_intent}")
print("---")
print(result_summary)

Original Question: What services are best for asynchornous communications?
Optimized Question: Inquiring about asynchronous communication services
---
Answer: Azure Service Bus and Azure Queue Storage are best for asynchronous communications.

Explanation: Azure Service Bus provides features like message queuing and publish-subscribe, which are essential for asynchronous communication. Azure Queue Storage is a message queuing service that enables you to build asynchronous solutions.
